In [3]:
import pandas as pd
import re
import ast
import time
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

nltk.download("punkt")

# 🕒 بدء المؤقت
start_time = time.time()

# 🧾 تحميل الملف
print("📂 Loading dataset...")
df = pd.read_csv("/Users/fellwakh/Downloads/newpar/books_and_genres.csv")

# حذف العمود الزائد لو موجود
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

print(f"✅ Dataset loaded successfully: {len(df)} rows\n")

# 🧹 تنظيف النصوص
def clean_text(text):
    text = re.sub(r"[^a-zA-Z\s]", " ", str(text))
    return text.lower()

df["clean_text"] = df["text"].apply(clean_text)

# 🎭 استخراج الأنواع الأدبية
def extract_genres(genres_str):
    try:
        genres = ast.literal_eval(genres_str)
        if isinstance(genres, (set, list)):
            return list(genres)
        else:
            return []
    except:
        return []

df["genre_list"] = df["genres"].apply(extract_genres)

# ===================================================
# 🏷️  TOP 10 MOST FREQUENT GENRES
# ===================================================
all_genres = [g for sublist in df["genre_list"] for g in sublist]
genre_freq = Counter(all_genres).most_common(10)

print("===============================================")
print("🏷️  TOP 10 MOST FREQUENT GENRES")
print("===============================================")
for g, c in genre_freq:
    print(f"  {g:<25} {c}")

# ===================================================
# 🔠 TOP 15 WORDS
# ===================================================
print("\n===============================================")
print("🌍 OVERALL TEXT ANALYSIS (All Books Combined)")
print("===============================================")

# جمع كل النصوص في نص واحد
all_text = " ".join(df["clean_text"])
all_words = all_text.split()

# 🔹 أكثر 15 كلمة شيوعًا
word_freq = Counter(all_words).most_common(15)

print("\n🔹 Top 15 Words:")
for w, c in word_freq:
    print(f"  {w:<20} {c}")

# ===================================================
# 🧮 TF-IDF (Overall Important Terms)
# ===================================================
print("\n===============================================")
print("📊 TF-IDF TOP TERMS (Overall Importance)")
print("===============================================")

vectorizer = TfidfVectorizer(stop_words="english", max_features=20)
tfidf_matrix = vectorizer.fit_transform(df["clean_text"])
tfidf_terms = vectorizer.get_feature_names_out()

for term in tfidf_terms:
    print("  ", term)

# 🕒 حساب الزمن المستغرق
end_time = time.time()
elapsed = end_time - start_time
minutes = int(elapsed // 60)
seconds = int(elapsed % 60)

print("\n===============================================")
print("⏱️  PROCESSING TIME SUMMARY")
print("===============================================")
print(f"Total rows processed: {len(df)}")
print(f"Elapsed time: {minutes} min {seconds} sec")


[nltk_data] Downloading package punkt to /Users/fellwakh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


📂 Loading dataset...
✅ Dataset loaded successfully: 10635 rows

🏷️  TOP 10 MOST FREQUENT GENRES
  fiction                   7268
  classics                  5467
  20th-century              3207
  non-fiction               3035
  literature                2936
  history                   2936
  historical-fiction        2632
  novels                    2366
  historical                2262
  romance                   2149

🌍 OVERALL TEXT ANALYSIS (All Books Combined)

🔹 Top 15 Words:
  the                  35067507
  of                   18516551
  and                  18392449
  to                   15395077
  a                    12816898
  in                   10117560
  i                    7692484
  that                 6994246
  he                   6969040
  was                  6543660
  it                   6263874
  his                  5327421
  with                 4536514
  you                  4466103
  as                   4446074

📊 TF-IDF TOP TERMS (Overall Importance)

In [1]:
import pandas as pd
import re
import ast
import time
import statistics
import psutil
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import threading

nltk.download("punkt")

# ===== Helper Functions =====
def clean_text(text):
    text = re.sub(r"[^a-zA-Z\s]", " ", str(text))
    return text.lower()

def extract_genres(genres_str):
    try:
        genres = ast.literal_eval(genres_str)
        if isinstance(genres, (set, list)):
            return list(genres)
        else:
            return []
    except:
        return []

# ===== Function to Monitor CPU Utilization =====
def monitor_cpu(stop_flag, usage_list):
    while not stop_flag.is_set():
        usage_list.append(psutil.cpu_percent(interval=0.5))

# ===== Benchmarking Loop =====
num_trials = 3
execution_times = []
cpu_usages = []

print("===============================================")
print("Sequential Benchmarking Results")
print("===============================================")

for i in range(num_trials):
    start = time.time()

    # Start CPU monitor thread
    stop_flag = threading.Event()
    usage_list = []
    monitor_thread = threading.Thread(target=monitor_cpu, args=(stop_flag, usage_list))
    monitor_thread.start()

    # --- Run your full pipeline here ---
    df = pd.read_csv("/Users/fellwakh/Downloads/newpar/books_and_genres.csv")
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])

    df["clean_text"] = df["text"].apply(clean_text)
    df["genre_list"] = df["genres"].apply(extract_genres)

    # Word frequency
    all_text = " ".join(df["clean_text"])
    all_words = all_text.split()
    word_freq = Counter(all_words).most_common(15)

    # TF-IDF
    vectorizer = TfidfVectorizer(stop_words="english", max_features=20)
    tfidf_matrix = vectorizer.fit_transform(df["clean_text"])
    tfidf_terms = vectorizer.get_feature_names_out()

    # Stop CPU monitor
    stop_flag.set()
    monitor_thread.join()

    end = time.time()
    elapsed = end - start
    avg_cpu = statistics.mean(usage_list) if usage_list else 0

    execution_times.append(elapsed)
    cpu_usages.append(avg_cpu)

    minutes = int(elapsed // 60)
    seconds = int(elapsed % 60)
    print(f"Trial {i+1}: {minutes} min {seconds} sec  |  Avg CPU: {avg_cpu:.1f}%")

# ===== Summary =====
avg_time = statistics.mean(execution_times)
avg_cpu_total = statistics.mean(cpu_usages)
rows = len(df)

for idx, (t, c) in enumerate(zip(execution_times, cpu_usages), 1):
    m, s = divmod(t, 60)
    

avg_m, avg_s = divmod(avg_time, 60)
print("-----------------------------------------------")
print(f"Average time over {num_trials} runs: {int(avg_m)} min {int(avg_s)} sec")
print(f"Average CPU utilization: {avg_cpu_total:.1f}%")
print(f"Total rows processed: {rows}")

[nltk_data] Downloading package punkt to /Users/fellwakh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sequential Benchmarking Results
Trial 1: 8 min 13 sec  |  Avg CPU: 11.2%
Trial 2: 14 min 13 sec  |  Avg CPU: 11.8%
Trial 3: 12 min 23 sec  |  Avg CPU: 12.9%
-----------------------------------------------
Average time over 3 runs: 11 min 36 sec
Average CPU utilization: 12.0%
Total rows processed: 10635


In [1]:
import pandas as pd
import re
import ast
import time
import sys
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import statistics

nltk.download("punkt", quiet=True)

# Store results for each trial
trial_results = []

def run_analysis_trial(trial_num):
    
    trial_start = time.time()
    timing = {}
    
    # Load dataset
    load_start = time.time()
    df = pd.read_csv("/Users/fellwakh/Downloads/newpar/books_and_genres.csv")
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])
    timing["load"] = time.time() - load_start
    
    # Clean texts
    clean_start = time.time()
    def clean_text(text):
        text = re.sub(r"[^a-zA-Z\s]", " ", str(text))
        return text.lower()
    df["clean_text"] = df["text"].apply(clean_text)
    timing["clean"] = time.time() - clean_start
    
    # Extract genres
    extract_start = time.time()
    def extract_genres(genres_str):
        try:
            genres = ast.literal_eval(genres_str)
            if isinstance(genres, (set, list)):
                return list(genres)
            else:
                return []
        except:
            return []
    df["genre_list"] = df["genres"].apply(extract_genres)
    timing["extract_genres"] = time.time() - extract_start
    
    # Top 10 genres
    genres_start = time.time()
    all_genres = [g for sublist in df["genre_list"] for g in sublist]
    genre_freq = Counter(all_genres).most_common(10)
    timing["genres_analysis"] = time.time() - genres_start
    
    # Top 15 words
    words_start = time.time()
    all_text = " ".join(df["clean_text"])
    all_words = all_text.split()
    word_freq = Counter(all_words).most_common(15)
    timing["words_analysis"] = time.time() - words_start
    
    # TF-IDF
    tfidf_start = time.time()
    vectorizer = TfidfVectorizer(stop_words="english", max_features=20)
    tfidf_matrix = vectorizer.fit_transform(df["clean_text"])
    tfidf_terms = vectorizer.get_feature_names_out()
    timing["tfidf"] = time.time() - tfidf_start
    
    timing["total"] = time.time() - trial_start
    
    return {
        "trial": trial_num,
        "rows": len(df),
        "timing": timing
    }


# RUN 3 TRIALS
print("=" * 60)
print("STARTING BENCHMARK: 3 TRIALS")
print("=" * 60)
print()

for trial in range(1, 4):
    print(f"Running Trial {trial}/3...")
    try:
        result = run_analysis_trial(trial)
        trial_results.append(result)
        print(f"  Trial {trial} completed in {result['timing']['total']:.4f}s")
    except Exception as e:
        print(f"   ❌ Trial {trial} failed: {e}")
        sys.exit(1)

print()
print("=" * 60)
print("BENCHMARK RESULTS - AVERAGE OF 3 TRIALS")
print("=" * 60)
print()

# Extract timing data for all trials
all_timings = {key: [] for key in trial_results[0]["timing"].keys()}
for result in trial_results:
    for key, value in result["timing"].items():
        all_timings[key].append(value)

# Calculate averages
avg_results = []
for task, times in all_timings.items():
    avg = statistics.mean(times)
    avg_results.append({
        "Task": task.upper(),
        "Average Time (s)": f"{avg:.4f}"
    })
    print(f"{task.upper():<20} {avg:.4f}s")

print()
print("=" * 30)
print("SUMMARY")
print("=" * 30)

avg_total = statistics.mean(all_timings["total"])
print(f"Total Rows Processed:  {trial_results[0]['rows']}")
print(f"Average Total Time:    {avg_total:.4f}s")
print("=" * 30)

STARTING BENCHMARK: 3 TRIALS

Running Trial 1/3...
  Trial 1 completed in 473.8689s
Running Trial 2/3...
  Trial 2 completed in 480.5118s
Running Trial 3/3...
  Trial 3 completed in 485.9123s

BENCHMARK RESULTS - AVERAGE OF 3 TRIALS

LOAD                 18.1296s
CLEAN                30.8228s
EXTRACT_GENRES       0.1474s
GENRES_ANALYSIS      0.0038s
WORDS_ANALYSIS       157.9542s
TFIDF                273.0392s
TOTAL                480.0977s

SUMMARY
Total Rows Processed:  10635
Average Total Time:    480.0977s
